# Integration & Infinite Series

**SIIEA Quantum Engineering Curriculum**
- **Curriculum Days:** 29-56
- **License:** CC BY-NC-SA 4.0 | Siiea Innovations, LLC

---

In [ ]:
# Hardware detection — adapts simulations to your machine
import sys, os
sys.path.insert(0, os.path.join(os.path.dirname(os.path.abspath("__file__")), ".."))
try:
    from hardware_config import HARDWARE, get_max_qubits
    print(f"Hardware: {HARDWARE['chip']} | {HARDWARE['memory_gb']} GB | Profile: {HARDWARE['profile']}")
    print(f"Max qubits: {get_max_qubits('safe')} (safe) / {get_max_qubits('max')} (max)")
except ImportError:
    print("hardware_config.py not found — using defaults")
    print("Run setup.sh from the repo root to generate it")

In [ ]:
# Standard scientific stack
import numpy as np
import math
import matplotlib.pyplot as plt
import sympy as sp
from sympy import (symbols, integrate, sin, cos, exp, log, pi, oo,
                   series, factorial, Rational, sqrt, simplify, latex,
                   apart, Piecewise, Sum)
from scipy import integrate as sci_integrate

%matplotlib inline

# Publication-quality defaults
plt.rcParams.update({
    'figure.figsize': (10, 6),
    'font.size': 12,
    'axes.labelsize': 14,
    'axes.titlesize': 15,
    'lines.linewidth': 2,
    'legend.fontsize': 11,
    'figure.dpi': 100,
})

sp.init_printing(use_unicode=True)
print('Libraries loaded successfully.')

## 1. Riemann Sums and the Definite Integral

The definite integral is defined as the limit of Riemann sums:

$$\int_a^b f(x)\,dx = \lim_{n\to\infty}\sum_{i=1}^{n} f(x_i^*)\,\Delta x$$

where $\Delta x = \frac{b-a}{n}$ and $x_i^*$ is a sample point in the $i$-th subinterval.

### Types of Riemann sums:
- **Left endpoint:** $x_i^* = a + (i-1)\Delta x$
- **Right endpoint:** $x_i^* = a + i\,\Delta x$
- **Midpoint:** $x_i^* = a + (i - \tfrac{1}{2})\Delta x$

As $n \to \infty$, all three converge to the same value for continuous functions.

In [ ]:
# Riemann sum visualization with increasing partitions
def riemann_sum_plot(f, a, b, partitions_list, f_label=r'$f(x) = x^2$'):
    '''Show Riemann sums converging to the integral.'''
    fig, axes = plt.subplots(1, len(partitions_list),
                              figsize=(5 * len(partitions_list), 5))
    if len(partitions_list) == 1:
        axes = [axes]

    x_fine = np.linspace(a, b, 500)

    # Exact integral for comparison (x^2 from 0 to 2 = 8/3)
    exact = sci_integrate.quad(f, a, b)[0]

    for ax, n in zip(axes, partitions_list):
        dx = (b - a) / n
        x_left = np.linspace(a, b - dx, n)  # Left endpoints
        heights = f(x_left)

        # Draw rectangles
        for i in range(n):
            rect = plt.Rectangle((x_left[i], 0), dx, heights[i],
                                  edgecolor='steelblue', facecolor='lightblue',
                                  alpha=0.7, linewidth=0.8)
            ax.add_patch(rect)

        # Riemann sum value
        rs = np.sum(heights * dx)

        ax.plot(x_fine, f(x_fine), 'r-', lw=2, label=f_label)
        ax.set_xlim(a - 0.1, b + 0.1)
        ax.set_ylim(0, f(b) * 1.15)
        ax.set_xlabel('x')
        ax.set_ylabel('f(x)')
        ax.set_title(f'n = {n}\nSum = {rs:.6f}')
        ax.legend(loc='upper left', fontsize=9)
        ax.grid(True, alpha=0.2)

    fig.suptitle(
        f'Left Riemann Sums -> Exact Integral = {exact:.6f}',
        fontsize=15, y=1.03
    )
    plt.tight_layout()
    plt.show()
    print(f'Exact integral value: {exact:.10f}')

riemann_sum_plot(lambda x: x**2, 0, 2, [4, 10, 50])

In [ ]:
# Convergence analysis: error vs number of partitions
def left_riemann(f, a, b, n):
    dx = (b - a) / n
    x_left = np.linspace(a, b - dx, n)
    return np.sum(f(x_left) * dx)

def mid_riemann(f, a, b, n):
    dx = (b - a) / n
    x_mid = np.linspace(a + dx/2, b - dx/2, n)
    return np.sum(f(x_mid) * dx)

exact_val = 8 / 3  # integral of x^2 from 0 to 2
ns = np.array([5, 10, 20, 50, 100, 200, 500, 1000])

left_errors = [abs(left_riemann(lambda x: x**2, 0, 2, n) - exact_val) for n in ns]
mid_errors = [abs(mid_riemann(lambda x: x**2, 0, 2, n) - exact_val) for n in ns]

fig, ax = plt.subplots(figsize=(9, 5))
ax.loglog(ns, left_errors, 'bo-', label='Left endpoint (error ~ 1/n)')
ax.loglog(ns, mid_errors, 'rs-', label='Midpoint (error ~ 1/n^2)')
ax.loglog(ns, 5/ns, 'b--', alpha=0.4, label=r'Reference $O(1/n)$')
ax.loglog(ns, 5/ns**2, 'r--', alpha=0.4, label=r'Reference $O(1/n^2)$')
ax.set_xlabel('Number of partitions (n)')
ax.set_ylabel('Absolute error')
ax.set_title('Riemann Sum Convergence Rates')
ax.legend()
ax.grid(True, alpha=0.3, which='both')
plt.tight_layout()
plt.show()
print('Midpoint rule converges quadratically -- much faster than left/right endpoint.')

## 2. The Fundamental Theorem of Calculus

The FTC connects differentiation and integration:

**Part 1 (FTC-1):** If $F(x) = \int_a^x f(t)\,dt$, then $F'(x) = f(x)$.

**Part 2 (FTC-2):** If $F$ is any antiderivative of $f$, then

$$\int_a^b f(x)\,dx = F(b) - F(a).$$

This is one of the most profound results in mathematics -- it links the
local concept of rate of change to the global concept of accumulation.

In [ ]:
# Fundamental Theorem of Calculus: symbolic + numerical verification
x, t = symbols('x t', real=True)

# Example: integral of sin(t) from 0 to x
F_symbolic = integrate(sin(t), (t, 0, x))
F_prime = sp.diff(F_symbolic, x)

print('FTC Part 1 Verification')
print('=' * 50)
print(f'  F(x) = integral of sin(t) dt from 0 to x = {F_symbolic}')
print(f"  F'(x) = {F_prime}")
print(f'  f(x) = sin(x)')
print(f"  F'(x) == f(x)? {simplify(F_prime - sin(x)) == 0}")

# FTC Part 2: definite integrals
print('\nFTC Part 2 -- Definite Integrals')
print('=' * 50)

integrals = [
    (x**2, 0, 3,     'x^2 from 0 to 3'),
    (sin(x), 0, pi,  'sin(x) from 0 to pi'),
    (exp(-x), 0, 1,  'e^(-x) from 0 to 1'),
    (1/(1+x**2), 0, 1, '1/(1+x^2) from 0 to 1'),
]

for expr, a_val, b_val, desc in integrals:
    symbolic_result = integrate(expr, (x, a_val, b_val))
    numerical_result = float(symbolic_result.evalf())
    # Verify with scipy
    f_num = sp.lambdify(x, expr, 'numpy')
    scipy_result = sci_integrate.quad(f_num, float(a_val), float(b_val))[0]

    print(f'\n  {desc}')
    print(f'    Symbolic: {symbolic_result} = {numerical_result:.10f}')
    print(f'    SciPy:    {scipy_result:.10f}')
    print(f'    Match:    {abs(numerical_result - scipy_result) < 1e-10}')

## 3. Integration Techniques with SymPy

### 3.1 Substitution (u-substitution)

$$\int f(g(x))\,g'(x)\,dx = \int f(u)\,du \quad \text{where } u = g(x)$$

### 3.2 Integration by parts

$$\int u\,dv = uv - \int v\,du$$

### 3.3 Partial fractions

$$\frac{P(x)}{Q(x)} = \frac{A_1}{x - r_1} + \frac{A_2}{x - r_2} + \cdots$$

SymPy's `integrate()` handles all these techniques automatically, but understanding
the method is essential for quantum mechanics where analytical solutions
are often needed.

In [ ]:
# Integration techniques demonstrated with SymPy
x = symbols('x')

print('INTEGRATION TECHNIQUES')
print('=' * 65)

# u-Substitution examples
print('\n--- u-Substitution ---')
sub_integrals = [
    (cos(x) * exp(sin(x)),        'cos(x) * e^(sin(x))'),
    (x * exp(x**2),                'x * e^(x^2)'),
    (2*x / (x**2 + 1),            '2x / (x^2 + 1)'),
]
for expr, desc in sub_integrals:
    result = integrate(expr, x)
    print(f'  integral of {desc:30s} = {result}')

# Integration by parts
print('\n--- Integration by Parts ---')
parts_integrals = [
    (x * exp(x),                   'x * e^x'),
    (x * sin(x),                   'x * sin(x)'),
    (x**2 * cos(x),                'x^2 * cos(x)'),
    (log(x),                       'ln(x)'),
]
for expr, desc in parts_integrals:
    result = integrate(expr, x)
    print(f'  integral of {desc:30s} = {result}')

# Partial fractions
print('\n--- Partial Fractions ---')
pf_integrals = [
    (1 / (x**2 - 1),               '1/(x^2 - 1)'),
    ((3*x + 5) / (x**2 + 4*x + 3), '(3x+5)/(x^2+4x+3)'),
    (1 / (x*(x+1)*(x+2)),          '1/(x(x+1)(x+2))'),
]
for expr, desc in pf_integrals:
    decomposed = apart(expr, x)
    result = integrate(expr, x)
    print(f'  {desc:30s}')
    print(f'    Partial fractions: {decomposed}')
    print(f'    Integral:          {result}')

## 4. Improper Integrals and Convergence

An **improper integral** has an infinite limit or an integrand with a
singularity:

$$\int_0^{\infty} e^{-x}\,dx = 1, \qquad \int_0^1 \frac{1}{\sqrt{x}}\,dx = 2$$

### Key convergence tests:
- **Comparison test:** If $0 \le f(x) \le g(x)$ and $\int g$ converges, then $\int f$ converges.
- **p-test:** $\int_1^{\infty} \frac{1}{x^p}\,dx$ converges iff $p > 1$.

### The Gaussian integral

$$\int_{-\infty}^{\infty} e^{-x^2}\,dx = \sqrt{\pi}$$

This integral appears throughout quantum mechanics (Gaussian wavefunctions).

In [ ]:
# Improper integrals -- symbolic and numerical
x = symbols('x')

print('IMPROPER INTEGRALS')
print('=' * 60)

improper = [
    (exp(-x),         (x, 0, oo),    'e^(-x) from 0 to inf'),
    (1/x**2,          (x, 1, oo),    '1/x^2 from 1 to inf'),
    (1/sqrt(x),       (x, 0, 1),     '1/sqrt(x) from 0 to 1'),
    (exp(-x**2),      (x, -oo, oo),  'e^(-x^2) from -inf to inf (Gaussian)'),
    (1/(1 + x**2),    (x, -oo, oo),  '1/(1+x^2) from -inf to inf'),
    (x * exp(-x),     (x, 0, oo),    'x*e^(-x) from 0 to inf'),
]

for expr, bounds, desc in improper:
    result = integrate(expr, bounds)
    print(f'  {desc:45s} = {result}')

# Visualize the Gaussian integral convergence
print(f'\nGaussian integral: sqrt(pi) = {float(sqrt(pi)):.10f}')

# Numerical verification
from scipy.integrate import quad
val, err = quad(lambda x: np.exp(-x**2), -100, 100)
print(f'SciPy numerical:             {val:.10f}')

# Plot the Gaussian and its cumulative integral
x_vals = np.linspace(-4, 4, 500)
gaussian = np.exp(-x_vals**2)

# Cumulative integral
cumulative = np.array([quad(lambda t: np.exp(-t**2), -4, xi)[0] for xi in x_vals])

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 5))

ax1.plot(x_vals, gaussian, 'b-', lw=2)
ax1.fill_between(x_vals, gaussian, alpha=0.2, color='blue')
ax1.set_xlabel('x')
ax1.set_ylabel(r'$e^{-x^2}$')
ax1.set_title(r'Gaussian: $e^{-x^2}$')
ax1.annotate(rf'Area = $\sqrt{{\pi}} \approx {np.sqrt(np.pi):.4f}$',
             xy=(0, 0.5), fontsize=13, ha='center',
             bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
ax1.grid(True, alpha=0.3)

ax2.plot(x_vals, cumulative, 'r-', lw=2)
ax2.axhline(np.sqrt(np.pi), color='gray', ls='--', alpha=0.5,
            label=rf'$\sqrt{{\pi}}$')
ax2.set_xlabel('x')
ax2.set_ylabel('Cumulative integral')
ax2.set_title(r'$\int_{-4}^{x} e^{-t^2}\,dt$')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Power Series, Taylor/Maclaurin with Error Bounds

A **power series** centered at $a$ is

$$\sum_{n=0}^{\infty} c_n (x - a)^n$$

with **radius of convergence** $R$ determined by the ratio test:

$$R = \lim_{n\to\infty}\left|\frac{c_n}{c_{n+1}}\right|$$

### Taylor remainder (Lagrange form)

For the $n$-th Taylor polynomial $T_n(x)$, the error satisfies:

$$|f(x) - T_n(x)| \le \frac{M_{n+1}}{(n+1)!}|x - a|^{n+1}$$

where $M_{n+1} = \max_{c\in[a,x]}|f^{(n+1)}(c)|$.

In [ ]:
# Taylor polynomial error bounds for e^x
x_sym = symbols('x')

# For e^x centered at 0: |R_n(x)| <= e^|x| * |x|^(n+1) / (n+1)!
# since all derivatives of e^x are e^x, and max on [0,x] is e^|x|

x_test = 1.0  # Evaluate error at x = 1
exact_val = np.exp(x_test)

orders = list(range(1, 16))
taylor_vals = []
actual_errors = []
bound_errors = []

for n in orders:
    # Taylor polynomial value
    taylor_sum = sum(x_test**k / math.factorial(k) for k in range(n + 1))
    taylor_vals.append(taylor_sum)

    actual_err = abs(exact_val - taylor_sum)
    actual_errors.append(actual_err)

    # Lagrange error bound: e^1 * 1^(n+1) / (n+1)!
    bound = np.exp(x_test) * x_test**(n+1) / math.factorial(n+1)
    bound_errors.append(bound)

fig, ax = plt.subplots(figsize=(10, 6))
ax.semilogy(orders, actual_errors, 'bo-', label='Actual error')
ax.semilogy(orders, bound_errors, 'r^--', label='Lagrange error bound')
ax.set_xlabel('Taylor polynomial order n')
ax.set_ylabel('Error (log scale)')
ax.set_title(r'Taylor Series Error for $e^x$ at $x = 1$')
ax.legend()
ax.grid(True, alpha=0.3, which='both')
plt.tight_layout()
plt.show()

# Print table
print(f'{"Order":>5s} {"Taylor Value":>18s} {"Actual Error":>14s} {"Bound":>14s}')
print('-' * 55)
for n, tv, ae, be in zip(orders, taylor_vals, actual_errors, bound_errors):
    print(f'{n:5d} {tv:18.12f} {ae:14.2e} {be:14.2e}')

## 6. Fourier Series Preview: Square Wave Approximation

A **Fourier series** represents a periodic function as a sum of sines and cosines:

$$f(x) = \frac{a_0}{2} + \sum_{n=1}^{\infty}\left[a_n\cos(nx) + b_n\sin(nx)\right]$$

The **square wave** (period $2\pi$) has Fourier series:

$$f(x) = \frac{4}{\pi}\sum_{k=0}^{\infty}\frac{\sin((2k+1)x)}{2k+1}$$

This is the foundation of **spectral methods** in quantum mechanics, where
we expand wavefunctions in an orthonormal basis of eigenstates.

In [ ]:
# Fourier series: square wave approximation
def square_wave_fourier(x, N):
    '''Approximate square wave using first N terms of Fourier series.'''
    result = np.zeros_like(x)
    for k in range(N):
        n = 2 * k + 1  # Odd harmonics only
        result += np.sin(n * x) / n
    return (4 / np.pi) * result

x_vals = np.linspace(-2 * np.pi, 2 * np.pi, 1000)

# Exact square wave for reference
square_exact = np.sign(np.sin(x_vals))

fig, axes = plt.subplots(2, 2, figsize=(13, 9))
terms_list = [1, 3, 10, 50]

for ax, N in zip(axes.flat, terms_list):
    ax.plot(x_vals, square_exact, 'k-', lw=1, alpha=0.4, label='Exact')
    ax.plot(x_vals, square_wave_fourier(x_vals, N), 'b-', lw=1.5,
            label=f'{N} term{"s" if N > 1 else ""}')
    ax.set_xlabel('x')
    ax.set_ylabel('f(x)')
    ax.set_title(f'Fourier Approximation: {N} term{"s" if N > 1 else ""}')
    ax.legend(loc='upper right', fontsize=9)
    ax.set_ylim(-1.5, 1.5)
    ax.grid(True, alpha=0.3)

plt.suptitle('Fourier Series: Square Wave Approximation', fontsize=15, y=1.01)
plt.tight_layout()
plt.show()
print('Notice the Gibbs phenomenon: overshoots near discontinuities persist')
print('even as the number of terms increases (converges to ~9% overshoot).')

## 7. Quantum Mechanics Connection

### Integrals in Quantum Mechanics

Integration is ubiquitous in QM. Three fundamental applications:

**1. Normalization:** A valid wavefunction must satisfy
$$\int_{-\infty}^{\infty} |\Psi(x)|^2\,dx = 1$$

**2. Expectation values:** The average measured value of an observable $\hat{A}$:
$$\langle\hat{A}\rangle = \int_{-\infty}^{\infty} \Psi^*(x)\,\hat{A}\,\Psi(x)\,dx$$

**3. Probability:** The probability of finding a particle in $[a, b]$:
$$P(a \le x \le b) = \int_a^b |\Psi(x)|^2\,dx$$

### The Gaussian Wavepacket

The most common wavefunction in QM is the Gaussian:

$$\Psi(x) = \left(\frac{1}{2\pi\sigma^2}\right)^{1/4} e^{-x^2/(4\sigma^2)}$$

In [ ]:
# QM Connection: Gaussian wavepacket -- normalization, probability, expectation
def gaussian_wavepacket(x, sigma=1.0, x0=0.0, k0=0.0):
    '''Gaussian wavepacket centered at x0 with momentum k0.'''
    norm = (1 / (2 * np.pi * sigma**2))**0.25
    return norm * np.exp(-(x - x0)**2 / (4 * sigma**2)) * np.exp(1j * k0 * x)

x_vals = np.linspace(-10, 10, 2000)
sigma = 1.0

psi = gaussian_wavepacket(x_vals, sigma=sigma, x0=0, k0=2.0)
prob_density = np.abs(psi)**2

# Verify normalization via numerical integration
dx = x_vals[1] - x_vals[0]
norm = np.trapezoid(prob_density, x_vals)

# Expectation value of x: <x> = integral of x * |psi|^2
x_expectation = np.trapezoid(x_vals * prob_density, x_vals)

# Expectation value of x^2
x2_expectation = np.trapezoid(x_vals**2 * prob_density, x_vals)

# Uncertainty (standard deviation)
delta_x = np.sqrt(x2_expectation - x_expectation**2)

# Probability of finding particle in [-sigma, sigma]
mask = (x_vals >= -sigma) & (x_vals <= sigma)
prob_in_sigma = np.trapezoid(prob_density[mask], x_vals[mask])

print('Gaussian Wavepacket Analysis')
print('=' * 50)
print(f'  sigma = {sigma}')
print(f'  Normalization:  integral |psi|^2 dx = {norm:.10f}  (should be 1)')
print(f'  <x>           = {x_expectation:.10f}  (should be 0)')
print(f'  <x^2>         = {x2_expectation:.10f}')
print(f'  Delta x       = {delta_x:.10f}  (should be {sigma*np.sqrt(2)/2:.10f})')
print(f'  P(-sigma < x < sigma) = {prob_in_sigma:.6f}  (~68.3%)')

# Plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 5))

ax1.plot(x_vals, np.real(psi), 'b-', alpha=0.7, label=r'Re($\Psi$)')
ax1.plot(x_vals, np.imag(psi), 'r-', alpha=0.7, label=r'Im($\Psi$)')
ax1.plot(x_vals, np.abs(psi), 'k--', lw=1.5, label=r'$|\Psi|$')
ax1.set_xlabel('x')
ax1.set_ylabel(r'$\Psi(x)$')
ax1.set_title('Gaussian Wavepacket (k0 = 2)')
ax1.legend()
ax1.grid(True, alpha=0.3)

ax2.plot(x_vals, prob_density, 'b-', lw=2)
ax2.fill_between(x_vals[mask], prob_density[mask], alpha=0.3, color='green',
                  label=rf'P($|x|<\sigma$) = {prob_in_sigma:.4f}')
ax2.axvline(x_expectation, color='red', ls='--',
            label=rf'$\langle x\rangle$ = {x_expectation:.2f}')
ax2.set_xlabel('x')
ax2.set_ylabel(r'$|\Psi(x)|^2$')
ax2.set_title('Probability Density')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.suptitle('Quantum Mechanics: Gaussian Wavepacket', fontsize=15, y=1.02)
plt.tight_layout()
plt.show()

## Summary

### Key Formulas

| Concept | Formula |
|---|---|
| Riemann sum | $\sum_{i=1}^n f(x_i^*)\Delta x \to \int_a^b f\,dx$ |
| FTC Part 1 | $\frac{d}{dx}\int_a^x f(t)\,dt = f(x)$ |
| FTC Part 2 | $\int_a^b f(x)\,dx = F(b) - F(a)$ |
| By parts | $\int u\,dv = uv - \int v\,du$ |
| Gaussian integral | $\int_{-\infty}^{\infty}e^{-x^2}dx = \sqrt{\pi}$ |
| Fourier series | $f(x) = \frac{a_0}{2}+\sum(a_n\cos nx + b_n\sin nx)$ |
| QM normalization | $\int|\Psi|^2 dx = 1$ |
| QM expectation | $\langle\hat{A}\rangle = \int\Psi^*\hat{A}\Psi\,dx$ |

### Main Takeaways

1. Riemann sums converge to the definite integral; midpoint rule converges faster ($O(1/n^2)$) than left/right ($O(1/n)$).
2. The Fundamental Theorem of Calculus links derivatives (local rates) with integrals (global accumulation).
3. Symbolic integration (SymPy) handles substitution, parts, and partial fractions; numerical integration (SciPy) provides verification.
4. Improper integrals -- especially the Gaussian integral -- appear throughout quantum mechanics.
5. Fourier series decompose functions into orthogonal components, just as quantum states decompose into energy eigenstates.
6. Integration underpins the three pillars of QM: normalization, expectation values, and probability.

---
*Next month: Multivariable Calculus & ODEs (Days 57-84)*